Name: Sathish Kumar Rajendiran
Task: Week7: MongoDB Class Activity 2
Date: 8/13/2020

In [1]:
#import libraries
from urllib import request
import json
import pymongo
from pymongo import MongoClient
from datetime import datetime
import sys
import pandas as pd
import os
import csv
print("all libraries imported successfully")

all libraries imported successfully


In [2]:
#verify current directory
print('working directory: ',os.getcwd())

working directory:  /Users/sathishrajendiran/ist652-python


In [3]:
ls *.csv

Border_Crossing_Entry_Data.csv*       price_of_gasoline.csv
albb.salaries.2003.Pitchers.csv       tv_life.csv
albb.salaries.2003.under-million.csv


In [4]:
# define file name
infile = 'Border_Crossing_Entry_Data.csv'
# Working with file, list and sorting
try: 
    data = pd.read_csv(infile)
    print("data has been processed \n")
    print('top 2 rows:\n',data.head(2))
    print('\nbottom 2 rows:\n',data.tail(2))
    
except:
    print("Is the file in correct directory?")

data has been processed 

top 2 rows:
   Port Name State  Port Code            Border            Date  \
0     Alcan    AK       3104  US-Canada Border  2/1/2020 00:00   
1     Alcan    AK       3104  US-Canada Border  2/1/2020 00:00   

                       Measure  Value  
0  Personal Vehicle Passengers   1414  
1            Personal Vehicles    763  

bottom 2 rows:
        Port Name State  Port Code            Border            Date  \
355509   Carbury    ND       3421  US-Canada Border  1/1/1996 00:00   
355510   Skagway    AK       3103  US-Canada Border  1/1/1996 00:00   

                       Measure  Value  
355509  Truck Containers Empty      0  
355510                   Buses      3  


In [5]:
#check index
data.index

RangeIndex(start=0, stop=355511, step=1)

In [6]:
# dropping null value columns to avoid errors 
data.dropna(inplace = True) 

In [7]:
# Change the column names 
data.columns = ['PortName','State','PortCode','Border','Date','Measure','Value']
data.head()

,PortName,State,PortCode,Border,Date,Measure,Value
0,Alcan,AK,3104,US-Canada Border,2/1/2020 00:00,Personal Vehicle Passengers,1414
1,Alcan,AK,3104,US-Canada Border,2/1/2020 00:00,Personal Vehicles,763
2,Alcan,AK,3104,US-Canada Border,2/1/2020 00:00,Truck Containers Empty,412
3,Alcan,AK,3104,US-Canada Border,2/1/2020 00:00,Truck Containers Full,122
4,Alcan,AK,3104,US-Canada Border,2/1/2020 00:00,Trucks,545


In [8]:
# storing dtype before converting 
before = data.dtypes
print('dtype before converting:\n',before)

dtype before converting:
 PortName    object
State       object
PortCode     int64
Border      object
Date        object
Measure     object
Value        int64
dtype: object


In [9]:
#convert Area and Population values to float
data['Date'] = data['Date'].astype('datetime64[ns]')
# data['Value'] = data['Value'].astype('float64')
data.head()

,PortName,State,PortCode,Border,Date,Measure,Value
0,Alcan,AK,3104,US-Canada Border,2020-02-01,Personal Vehicle Passengers,1414
1,Alcan,AK,3104,US-Canada Border,2020-02-01,Personal Vehicles,763
2,Alcan,AK,3104,US-Canada Border,2020-02-01,Truck Containers Empty,412
3,Alcan,AK,3104,US-Canada Border,2020-02-01,Truck Containers Full,122
4,Alcan,AK,3104,US-Canada Border,2020-02-01,Trucks,545


In [10]:
# storing dtype before converting 
after = data.dtypes

# printing to compare 
print('Before Conversion:\n', before) 
print('\nAfter Conversion:\n', after) 

Before Conversion:
 PortName    object
State       object
PortCode     int64
Border      object
Date        object
Measure     object
Value        int64
dtype: object

After Conversion:
 PortName            object
State               object
PortCode             int64
Border              object
Date        datetime64[ns]
Measure             object
Value                int64
dtype: object


In [11]:
#data frame shape
data.shape

(355511, 7)

In [12]:
# the url request was successful - convert the response to a string
data_json = json.loads(data.to_json(orient='records'))

In [13]:
print(data_json[:10])

[{'PortName': 'Alcan', 'State': 'AK', 'PortCode': 3104, 'Border': 'US-Canada Border', 'Date': 1580515200000, 'Measure': 'Personal Vehicle Passengers', 'Value': 1414}, {'PortName': 'Alcan', 'State': 'AK', 'PortCode': 3104, 'Border': 'US-Canada Border', 'Date': 1580515200000, 'Measure': 'Personal Vehicles', 'Value': 763}, {'PortName': 'Alcan', 'State': 'AK', 'PortCode': 3104, 'Border': 'US-Canada Border', 'Date': 1580515200000, 'Measure': 'Truck Containers Empty', 'Value': 412}, {'PortName': 'Alcan', 'State': 'AK', 'PortCode': 3104, 'Border': 'US-Canada Border', 'Date': 1580515200000, 'Measure': 'Truck Containers Full', 'Value': 122}, {'PortName': 'Alcan', 'State': 'AK', 'PortCode': 3104, 'Border': 'US-Canada Border', 'Date': 1580515200000, 'Measure': 'Trucks', 'Value': 545}, {'PortName': 'Alexandria Bay', 'State': 'NY', 'PortCode': 708, 'Border': 'US-Canada Border', 'Date': 1580515200000, 'Measure': 'Bus Passengers', 'Value': 1174}, {'PortName': 'Alexandria Bay', 'State': 'NY', 'PortCod

In [14]:
print('Border Crossing List', len(data_json))

Border Crossing List 355511


In [19]:
# Connection to Mongo DB
try:
    client = MongoClient('localhost', 27017)
    print ("Connected successfully!!!")
except pymongo.errors.ConnectionFailure as e:
    print ("Could not connect to MongoDB: %s" % e )

else:
    
    #drop db
    borderdb.borderentries.drop()
    
    # use database named usgs or create it if not there already
    borderdb = client['borderentry']
    # create collection named borderentries or create it if not there already
    bordercoll = borderdb['borderentries']
    # add all the border entries to the list
    bordercoll.insert_many(data_json)
    print("Added", len(data_json), "to Border Crossing entries collection in borderentry database")
    # close the database connection
    client.close()

Connected successfully!!!
Added 355511 to Border Crossing entries collection in borderentry database


In [20]:
#search the first item from the collection
bordercoll.find_one()

{'_id': ObjectId('5f35f5adb253933abdb60230'),
 'PortName': 'Alcan',
 'State': 'AK',
 'PortCode': 3104,
 'Border': 'US-Canada Border',
 'Date': 1580515200000,
 'Measure': 'Personal Vehicle Passengers',
 'Value': 1414}

In [21]:
#print the number of docs from db
print('Total Number of Documents: ',bordercoll.count_documents({}))

Total Number of Documents:  355511


In [22]:
# Aggregation 
cursor = bordercoll.aggregate([{"$group": 
       {"_id":"$Measure",
       "Crossings":{"$sum":'$Value'}}}]) 
  
for document in cursor: 
       print(document)

{'_id': 'Truck Containers Full', 'Crossings': 185463194}
{'_id': 'Bus Passengers', 'Crossings': 146027374}
{'_id': 'Buses', 'Crossings': 8754394}
{'_id': 'Personal Vehicles', 'Crossings': 2651535415}
{'_id': 'Rail Containers Empty', 'Crossings': 22386399}
{'_id': 'Truck Containers Empty', 'Crossings': 67036035}
{'_id': 'Pedestrians', 'Crossings': 1090067964}
{'_id': 'Personal Vehicle Passengers', 'Crossings': 5629526756}
{'_id': 'Trucks', 'Crossings': 264731943}
{'_id': 'Rail Containers Full', 'Crossings': 40492650}
{'_id': 'Trains', 'Crossings': 933270}
{'_id': 'Train Passengers', 'Crossings': 6472717}


In [23]:
#search the first item from the collection
bordercoll.find_one()

{'_id': ObjectId('5f35f5adb253933abdb60230'),
 'PortName': 'Alcan',
 'State': 'AK',
 'PortCode': 3104,
 'Border': 'US-Canada Border',
 'Date': 1580515200000,
 'Measure': 'Personal Vehicle Passengers',
 'Value': 1414}

In [24]:
#list db names
borderdb.list_collection_names()

['borderentries']

In [25]:
#drop db
borderdb.borderentries.drop()

In [26]:
#empty database
borderdb.list_collection_names()

[]